# SONATA Condition Modifications

This notebook demonstrates how to use SONATA **condition modifications** in BlueCelluLab.

Modifications are defined in the `conditions` block of a SONATA simulation config and allow you to
alter cell properties before the simulation runs. BlueCelluLab supports all five SONATA modification types:

| Type | Description |
|------|-------------|
| `ttx` | Block Na channels (insert TTXDynamicsSwitch) |
| `configure_all_sections` | Apply a statement to all sections of target cells |
| `section_list` | Apply a statement to a named section list (somatic, basal, apical, axonal) |
| `section` | Apply a statement to specific named sections (e.g. apic[10]) |
| `compartment_set` | Apply a statement to segments defined by a compartment set |

See the [SONATA-extension documentation](https://sonata-extension.readthedocs.io/en/latest/sonata_simulation.html#parameters-required-for-modifications) for the full specification.

## 1. Compile mechanisms

As in the previous tutorial, we first compile the NEURON mechanisms.

In [ ]:
!nrnivmodl ../mechanisms

In [ ]:
import json
import logging
from pathlib import Path

from matplotlib import pyplot as plt
import seaborn as sns

sns.set_style("white")

from bluecellulab import CircuitSimulation
from bluecellulab.circuit.config import SonataSimulationConfig

In [ ]:
# Enable INFO logging for the modifications module to see what happens
logging.basicConfig(level=logging.WARNING)
logging.getLogger("bluecellulab.simulation.modifications").setLevel(logging.INFO)

## 2. Examine the simulation config with modifications

We have prepared a simulation config that includes three modifications in its `conditions` block.

In [ ]:
sim_config_path = Path("sim_quick_scx_sonata_multicircuit") / "simulation_config_modifications.json"

with open(sim_config_path) as f:
    config_dict = json.load(f)

print(json.dumps(config_dict["conditions"], indent=4))

The three modifications are:

1. **`TTX_block_NodeB`** (`ttx`): Blocks Na channels on all cells in the `Mosaic_B` node set by inserting `TTXDynamicsSwitch`.
2. **`double_cm_all`** (`configure_all_sections`): Doubles membrane capacitance (`cm = 2.0`) on **all** sections of `Mosaic_A` cells.
3. **`scale_soma_cm`** (`section_list`): Further scales `cm` by 1.5x on only the **somatic** sections of `Mosaic_A` cells.

After both modifications, somatic sections of Mosaic_A cells will have `cm = 2.0 * 1.5 = 3.0`, while dendritic/axonal sections will have `cm = 2.0`.

## 3. Parse modifications with BlueCelluLab

BlueCelluLab parses the modifications into typed dataclasses via `get_modifications()`.

In [ ]:
sonata_config = SonataSimulationConfig(sim_config_path)
modifications = sonata_config.get_modifications()

for mod in modifications:
    print(f"  {mod.name}: type={mod.type}, class={type(mod).__name__}")
    if hasattr(mod, 'section_configure'):
        print(f"    section_configure: {mod.section_configure}")
    if hasattr(mod, 'node_set'):
        print(f"    node_set: {mod.node_set}")

## 4. Run simulation with modifications

When `CircuitSimulation.instantiate_gids()` is called, modifications are automatically applied after cells are created but before synapses and stimuli are added. This matches the ordering used by [neurodamus](https://github.com/openbraininstitute/neurodamus).

The INFO-level logs show exactly which modifications were applied and to how many sections/cells.

In [ ]:
sim = CircuitSimulation(sim_config_path)

In [ ]:
from bluepysnap import Simulation as snap_sim
import pandas as pd

snap_access = snap_sim(sim_config_path)
all_nodes = pd.concat([x[1] for x in snap_access.circuit.nodes.get()])
all_cell_ids = all_nodes.index.to_list()
print(f"All cells in circuit: {all_cell_ids}")

In [ ]:
# Instantiate all cells — modifications are applied automatically
sim.instantiate_gids(all_cell_ids, add_stimuli=False, add_synapses=False)

## 5. Verify modifications were applied

Let's inspect the cells to confirm the modifications took effect.

In [ ]:
print("=" * 70)
print("Checking membrane capacitance (cm) on Mosaic_A cells")
print("Expected: somatic cm=3.0 (2.0 * 1.5), dendritic cm=2.0")
print("=" * 70)

for cell_id in sim.cells:
    cell = sim.cells[cell_id]
    print(f"\nCell {cell_id}:")

    # Check soma cm
    if cell.somatic:
        soma_sec = cell.somatic[0]
        print(f"  soma cm = {soma_sec.cm}")

    # Check a dendritic section cm (if available)
    for sec_name, sec in list(cell.sections.items())[:5]:
        if 'dend' in sec_name or 'apic' in sec_name:
            print(f"  {sec_name} cm = {sec.cm}")
            break

## 6. Compare: with vs without modifications

To see the effect of modifications on simulation output, let's run two simulations
side by side — one with and one without modifications — and compare the voltage traces.

In [ ]:
# Simulation WITH modifications (already instantiated above)
sim.run(t_stop=100.0)
traces_with_mods = {}
for cell_id in sim.cells:
    traces_with_mods[cell_id] = {
        "time": sim.get_time_trace(),
        "voltage": sim.get_voltage_trace(cell_id),
    }

In [ ]:
# Simulation WITHOUT modifications (baseline)
baseline_config = Path("sim_quick_scx_sonata_multicircuit") / "simulation_config_noinput.json"
sim_baseline = CircuitSimulation(baseline_config)
sim_baseline.instantiate_gids(all_cell_ids, add_stimuli=False, add_synapses=False)
sim_baseline.run(t_stop=100.0)
traces_baseline = {}
for cell_id in sim_baseline.cells:
    traces_baseline[cell_id] = {
        "time": sim_baseline.get_time_trace(),
        "voltage": sim_baseline.get_voltage_trace(cell_id),
    }

In [ ]:
# Plot comparison
fig, axes = plt.subplots(len(sim.cells), 1, figsize=(12, 3 * len(sim.cells)), sharex=True)
if len(sim.cells) == 1:
    axes = [axes]

for ax, cell_id in zip(axes, sim.cells):
    if cell_id in traces_baseline:
        ax.plot(traces_baseline[cell_id]["time"], traces_baseline[cell_id]["voltage"],
                label="baseline", alpha=0.8)
    if cell_id in traces_with_mods:
        ax.plot(traces_with_mods[cell_id]["time"], traces_with_mods[cell_id]["voltage"],
                label="with modifications", alpha=0.8, linestyle="--")
    ax.set_ylabel("Voltage (mV)")
    ax.set_title(str(cell_id))
    ax.legend(loc="upper right")

axes[-1].set_xlabel("Time (ms)")
plt.tight_layout()
plt.savefig("ex2_modifications_comparison.pdf")
plt.show()

## 7. Graceful skipping behavior

An important design decision: modifications **skip** sections or cells that don't match,
rather than failing. This is essential because `node_set` targets can contain heterogeneous
cells (e.g., some with apical dendrites, some without).

- Sections missing a referenced attribute are silently skipped
- If zero sections matched, a warning is logged
- Invalid `section_configure` syntax still raises an error

This matches [neurodamus's behavior](https://github.com/openbraininstitute/neurodamus) for `configure_all_sections`
and extends it consistently to the new types (`section_list`, `section`, `compartment_set`).

## Summary

- Modifications are defined in `conditions.modifications` of the SONATA simulation config
- BlueCelluLab parses them via `SonataSimulationConfig.get_modifications()`
- They are automatically applied during `CircuitSimulation.instantiate_gids()`, after cell creation
- All five types are supported: `ttx`, `configure_all_sections`, `section_list`, `section`, `compartment_set`
- Logging at INFO level shows what was applied; WARNING level flags zero-match cases